In [41]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

[![](https://mermaid.ink/img/pako:eNqVktFqwyAUhl9FzlUG7Qtk0Jvmchmha3clhDM9a6VRgzGB0fbdp6YrW7LB6o1Hv_-I_885gbCSIIe9w_bAtsUjNywsYY0n47P1uD9crzUqU0s1ZGUoWKGGL7AnX7fOyl74WqCTWTUe2Docvov8UBvUlG1f2XPYJ6i1TSSVbSZAoxMYURmLaZdTIr1YxWICe9d0ke02T11EP_yx5XJ1MzXxGNl5Q4Le6Dzz94ftWc_V7jyA35TB_f-EKY07vpACukMfM0tRwQI0uZCIDCNyig9w8AfSxCEPpUR35MDNJeiw9_blwwjI37HpaAF9K9FToTCMlr7dklTeunIcujR7l0_dT9sK?type=png)](https://mermaid.live/edit#pako:eNqVktFqwyAUhl9FzlUG7Qtk0Jvmchmha3clhDM9a6VRgzGB0fbdp6YrW7LB6o1Hv_-I_885gbCSIIe9w_bAtsUjNywsYY0n47P1uD9crzUqU0s1ZGUoWKGGL7AnX7fOyl74WqCTWTUe2Docvov8UBvUlG1f2XPYJ6i1TSSVbSZAoxMYURmLaZdTIr1YxWICe9d0ke02T11EP_yx5XJ1MzXxGNl5Q4Le6Dzz94ftWc_V7jyA35TB_f-EKY07vpACukMfM0tRwQI0uZCIDCNyig9w8AfSxCEPpUR35MDNJeiw9_blwwjI37HpaAF9K9FToTCMlr7dklTeunIcujR7l0_dT9sK)

In [42]:
BASE_URL = 'https://www.buscape.com.br/tv' #url base
def requisition(page:int,base_url:str) -> bytes:
    '''
    ::DESC:: responsavel por requisitar e verificar se o site esta no ar

    ::PARAM:: 
        - page: pagina que raspara os dados
    ::RETURN::
        - conteudo da pagina
    '''

    url = f'{base_url}?page={page}' #concatena a page com a pagina em quetao

    r = requests.get(url)

    if r.ok:
        print("Site No Ar!")
        return r.content
    else:
        print("Site Fora do Ar!")


In [10]:
def get_main_div(content:bytes):
    '''
    ::DESC:: pega a div principal, que sera reponsavel por conter todas as infos que precisamos

    ::PARAM:: 
        - content: conteudo puro html da pagina
    ::RETURN::
        - main_div: div principal que precisaremos para retirar todos os dados
    '''

    soup  = BeautifulSoup(content,'html.parser')

    #main_div é uma lista que por sua vez vai possuir entre os seus elementos todos as subdivs
    main_div = soup.find_all('div',attrs={'class': 'Paper_Paper__4XALQ Paper_Paper__bordered__cl5Rh Card_Card__Zd8Ef Card_Card__clicable__ewI68 ProductCard_ProductCard__WWKKW'})

    return main_div

In [14]:
def get_product_card(main_div) -> list:
    '''
    ::DESC:: responsavel por entrar em cada cartão de cada anuncio do site

    ::PARAM:: 
        - main_div: div principal da pagina
    ::RETURN::
        - product_card: anuncio do produto contendo diversas informacoes importantes
    '''
    #entrando no "cartao" de cada produto
    product_card = []
    for i,v in enumerate(main_div):
        product_card.append(main_div[i].find_all('div',class_='ProductCard_ProductCard_Body__bnVUn'))

    return product_card

In [19]:
def get_tv_urls(main_div) -> list:
    '''
    ::DESC:: responsavel por minerar as urls de cada anuncio

    ::PARAM:: 
        - main_div: div principal da pagina
    ::RETURN::
        - lst_urls: lista com todos os links dos anuncios
    '''


    lst_urls = []
    for i,v in enumerate(main_div):
        anchor = main_div[i].find_all(('a'),class_='ProductCard_ProductCard_Inner__gapsh')[0] #entra em cada anchor
        href = anchor['href'] #puxa o href de cada (link da tv)

        url = BASE_URL[:-3] + href #concatena a url base com o link raspado

        lst_urls.append(url)

    return lst_urls

In [25]:
def get_tv_name(product_card) -> list:
    '''
    ::DESC:: responsavel por minerar os nomes das tvs cada anuncio

    ::PARAM:: 
        - product_card: card do anuncio da tv em questao
    ::RETURN::
        - lst_names: lista com todos os nomes das tvs dos anuncios
    '''

    lst_names = []
    # retirando o nome da TV no anuncio
    for i,v in enumerate(product_card):
        # dentro de cada cartao, entramos na div que possui o nome dela
        name_obj = product_card[i][0].find_all('div', class_='ProductCard_ProductCard_NameWrapper__45Z01') # objeto que possui o nome
        name = name_obj[0].text # nome de fato

        lst_names.append(name)
    return lst_names

In [26]:
def get_tv_price(product_card) -> list:
    '''
    ::DESC:: responsavel por minerar os precos das tvs cada anuncio

    ::PARAM:: 
        - product_card: card do anuncio da tv em questao
    ::RETURN::
        - lst_price: lista com todos os precos das tvs dos anuncios
    '''

    lst_price = []
    for i,v in enumerate(product_card):
        price_obj = product_card[i][0].find_all('p',class_='Text_Text__ARJdp Text_MobileHeadingS__HEz7L') #puxamos o paragrafo onde o texto esta
        lst_price.append(price_obj[0].text) #retiramos somente o texto do paragrafo

    return lst_price

In [28]:
def get_tv_pol(lst_names:list) -> list:
    '''
    ::DESC:: responsavel responsavel por retirar a polegada da TV em questao atraves do nome do anuncio
            - vamos usar o " como delimitador e supomos que as polegadas têm 2 algarismos
    ::PARAM:: 
        - lst_names: lista com os nomes dos anuncios
    ::RETURN::
        - lst_pol: lista com todos as polegadas das tvs dos anuncios
    '''
    lst_pol = []
    for i in lst_names:
        delimitador = i.find('"') #procutamos o delimitador " que simboliza a polegada
        lst_pol.append(i[(delimitador - 2):delimitador]) # fatiamos o elemento do delimitador até -2 elementos (supondo polegadas de 2 algarismos)

    return lst_pol




In [34]:
def get_tv_marca(lst_names:list) -> list:
    '''
    ::DESC::  responsavel por retirar a marca da TV em questao atraves do nome do anuncio
            - usaremos o " como delimitador e o espaço em branco como delimitador final
    ::PARAM:: 
        - lst_names: lista com os nomes dos anuncios
    ::RETURN::
        - lst_marcas: lista com todos as marcas das tvs dos anuncios
    '''


    lst_marcas = []
    for i in lst_names: 
        #toma o " como delimitador e soma + 2 para o encaixar ao index 
        first_delimitador = (i.find('"'))+2

        # fatia os elementos partindo do delimitador e então usa o 'partition' para separar em 3 e retirar o primeiro elemento da tupla gerada
        marca = i[first_delimitador:].partition(' ')[0]

        lst_marcas.append(marca)
    return lst_marcas

In [37]:
def schema(lst_names,lst_price,lst_pol,lst_marcas,lst_urls):
    schema = {'desc': lst_names,
      'price': lst_price,
      'pol':lst_pol,
      'manufacturer':lst_marcas,
      'url':lst_urls}
    
    return pd.DataFrame(schema)


In [39]:
#content = requisition(1,base_url=BASE_URL)
main_div = get_main_div(content)
product_card = get_product_card(main_div)
url = get_tv_urls(main_div)
names = get_tv_name(product_card)
price = get_tv_price(product_card)
pol = get_tv_pol(names)
marca = get_tv_marca(names)

In [43]:
schema(names,price,pol,marca,url)

,desc,price,pol,manufacturer,url
0,"Smart TV LED 50"" Samsung Crystal 4K HDR UN50CU...","R$ 2.159,00",50,Samsung,https://www.buscape.com.br/tv/smart-tv-led-50-...
1,"Smart TV LED 43"" Samsung Crystal 4K HDR UN43CU...","R$ 1.799,10",43,Samsung,https://www.buscape.com.br/tv/smart-tv-led-43-...
2,"Smart TV LED 65"" Samsung Crystal 4K HDR UN65CU...","R$ 3.112,71",65,Samsung,https://www.buscape.com.br/tv/smart-tv-led-65-...
3,"Smart TV LED 50"" Samsung Crystal 4K HDR UN50CU...","R$ 2.259,00",50,Samsung,https://www.buscape.com.br/tv/smart-tv-led-50-...
4,"Smart TV LED 32"" HQ HQ324 (139)","R$ 849,00",32,HQ,https://www.buscape.com.br/tv/smart-tv-led-32-...
5,"Smart TV LED 43"" Samsung Crystal 4K HDR UN43CU...","R$ 1.789,00",43,Samsung,https://www.buscape.com.br/tv/smart-tv-led-43-...
6,"Smart TV LED 55"" LG ThinQ AI 4K HDR 55UR8750PSA","R$ 2.479,00",55,LG,https://www.buscape.com.br/tv/smart-tv-lcd-55-...
7,"Smart TV LED 50"" TCL 4K HDR 50P635","R$ 1.749,44",50,TCL,https://www.buscape.com.br/tv/smart-tv-led-50-...
8,"Smart TV LED 65"" Samsung Crystal 4K HDR UN65CU...","R$ 3.199,98",65,Samsung,https://www.buscape.com.br/tv/smart-tv-led-65-...
9,"Smart TV QLED 55"" TCL 4K HDR 55C645","R$ 2.469,05",55,TCL,https://www.buscape.com.br/tv/smart-tv-qled-55...


content -> main_div -> [get_product_card(main_div) -> [get_tv_name(product_card) -> [get_tv_pol(lst_names), get_tv_marca(lst_names)] , get_tv_price(product_card)] , get_tv_urls(main_div)]

# TO-DO:
- escalar para mais de uma pagina
- fazer com que as listas se appendem persistindo no valor da pagina passada
